<a href="https://colab.research.google.com/github/Riccardo-Venturi/Tesi_Script_Colab/blob/main/Data_Augmentation_Factory1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perché la Cella 2 è lenta
a) Copia da Drive

Il file‑system FUSE di Drive ha throughput ballerino (spesso 5–20 MB/s) e latenza alta
GitHub
. Copiare centinaia di PNG da 512 px richiede parecchi secondi – ed è lineare con il numero di file. SUGGERIMENTO: esegui la cella una sola volta, poi lascia il dataset ridimensionato su disco locale di Colab o, meglio, su una cartella Drive già preridimensionata e salta la copia con la “cella 2.5” che hai citato
Reddit
.
b) cv2.resize in loop Python

cv2.resize è veloce in C++, ma fare 1000 chiamate dal loop Python resta costoso; è normale perdere 0.3–0.6 s/immagine
Stack Overflow
. Se devi rifarlo spesso, usa joblib.Parallel o una semplice lista di path e multiprocessing (l’I/O in cache lo regge) per saturare la CPU.

questa cella fa' quello
# ===================================================================
# CELLA 1: SETUP E CARICAMENTO DATI 512px
# ===================================================================
print("--- [1/4] Setup e Caricamento Dati 512px ---")
!pip install -q segmentation-models-pytorch==0.3.3 albumentations torchinfo
import torch, cv2, json, pathlib, numpy as np
import segmentation_models_pytorch as smp
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn, torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm.notebook import tqdm

print("✅ Librerie pronte.")

# Percorso del dataset GIÀ RIDIMENSIONATO a 512px su Drive
DATA_512_DRIVE_PATH = "/content/drive/MyDrive/Colab_Datasets/Augmented_Dataset_55" # Assumo sia questo
LOCAL_DATA_PATH = pathlib.Path("/content/dataset_512")

if not LOCAL_DATA_PATH.exists():
    print(f"Copiando dati da '{DATA_512_DRIVE_PATH}'...")
    !cp -r "{DATA_512_DRIVE_PATH}" "{LOCAL_DATA_PATH}"
    print("✅ Copia completata.")
else:
    print(f"ℹ️ Dataset già presente in '{LOCAL_DATA_PATH}'.")

In [ ]:
#@title #### Cella 1 Riferimento: Import e Definizione della Funzione VIENE USATO PER MODULARIZZARE IL TUTTO NEGLI ALtrI SCRIPT COLAB

#Questa cella conterrà **solo** le definizioni, senza eseguire nulla.

# ===================================================================
# CELLA 1: MODULO DI AUGMENTATION OFFLINE
# ===================================================================
# Questo notebook, quando importato, espone la funzione 'create_augmented_dataset'

import cv2, json, pathlib, numpy as np
import albumentations as A
from tqdm.notebook import tqdm
import shutil

print("Modulo 'Data_Augmentation_Factory' caricato.")

def create_augmented_dataset(
    original_zip_path: str,
    drive_save_path: str,
    n_augmentations_per_image: int = 15,
    image_size: int = 512,
    train_split_ratio: float = 0.75,
    val_split_ratio: float = 0.15,
    random_seed: int = 42
):
    """
    Funzione completa per creare un dataset augmentato offline.
    Prende uno zip, lo processa e salva il risultato su Google Drive.

    Args:
        original_zip_path (str): Percorso del file .zip con i dati originali.
        drive_save_path (str): Percorso su Google Drive dove salvare il dataset finale.
        ... (altri parametri con valori di default)

    Returns:
        pathlib.Path: Il percorso su Drive dove è stato salvato il dataset.
    """

    # Definisci i percorsi locali temporanei
    original_data_local = pathlib.Path("/content/original_dataset_temp")
    aug_data_local = pathlib.Path("/content/augmented_dataset_temp")

    # Pulisci le cartelle temporanee se esistono da una run precedente
    if original_data_local.exists():
        shutil.rmtree(original_data_local)
    if aug_data_local.exists():
        shutil.rmtree(aug_data_local)

    # --- 1. PREPARA I DATI ORIGINALI ---
    original_data_local.mkdir()
    print(f"Scompatto '{original_zip_path}'...")
    !unzip -q -o "{original_zip_path}" -d "{original_data_local}"
    print("✅ Dati originali pronti.")

    original_img_dir = original_data_local / "images"
    original_msk_dir = original_data_local / "masks"

    # --- 2. DEFINISCI LE PIPELINE DI AUGMENTATION ---
    # ... (il codice per definire train_aug_pipeline e val_test_aug_pipeline rimane identico) ...
    train_aug_pipeline = A.Compose([...]) # Inserisci la pipeline completa qui
    val_test_aug_pipeline = A.Compose([A.Resize(image_size, image_size)])

    # --- 3. SPLIT DEI DATI ORIGINALI ---
    # ... (il codice per lo split rimane identico) ...
    all_stems = sorted([...])
    np.random.seed(random_seed)
    # ... etc ...

    # --- 4. GENERAZIONE DEL NUOVO DATASET ---
    # ... (tutta la logica di generazione delle immagini rimane identica) ...

    # --- 5. SALVA IL NUOVO SPLIT FILE ---
    final_split = {"train": new_train_ids, "val": val_ids, "test": test_ids}
    with open(aug_data_local / "split.json", "w") as f:
        json.dump(final_split, f, indent=4)

    # --- 6. SALVA SU GOOGLE DRIVE ---
    drive_save_path = pathlib.Path(drive_save_path)
    drive_save_path.parent.mkdir(parents=True, exist_ok=True)
    if drive_save_path.exists():
        shutil.rmtree(drive_save_path) # Rimuovi la vecchia versione per evitare conflitti

    print(f"Salvataggio del dataset augmentato su Google Drive in: '{drive_save_path}'...")
    !cp -r "{aug_data_local}" "{drive_save_path}"
    print("✅ Salvataggio completato!")

    # Pulisci lo spazio locale di Colab
    shutil.rmtree(original_data_local)
    shutil.rmtree(aug_data_local)

    return drive_save_path


In [ ]:
#@title cella autodefinita codice 0.1 adessio si modula#### Cella 1: Setup e Parametri
'''
#Qui definiamo tutti i parametri del processo: percorsi, numero di augmentations, ecc.

# ===================================================================
# CELLA 1: SETUP E PARAMETRI DELLA FABBRICA DI DATI
# ===================================================================
print("--- Fabbrica di Dati Attivata ---")

# --- Installazioni e Import ---
!pip install -q albumentations opencv-python-headless tqdm
!pip install unzip
import cv2, json, pathlib, numpy as np
import albumentations as A
from tqdm.notebook import tqdm

print("✅ Librerie pronte.")

# --- PARAMETRI DI CONFIGURAZIONE ---

# 1. Percorsi dei dati originali (da dove leggere)
ORIGINAL_ZIP_PATH = "/content/drive/MyDrive/DatasetUnet/unet_dataset_multiclasse(1).zip"
!unzip -q -o "/content/drive/MyDrive/DatasetUnet/unet_dataset_multiclasse(1)" -d "/content/original_dataset"

ORIGINAL_DATA_LOCAL = pathlib.Path("/content/original_dataset")

# 2. Percorsi del nuovo dataset augmentato (dove salvare)
AUG_DATA_LOCAL = pathlib.Path("/content/augmented_dataset_v2AUG")
DRIVE_SAVE_PATH = pathlib.Path("/content/drive/MyDrive/Colab_Datasets/Augmented_Dataset_v2AUG")

# 3. Parametri di Augmentation e Split
N_AUGMENTATIONS_PER_IMAGE = 15  # Aumentiamo ancora un po'
IMAGE_SIZE = 704                # Dimensione finale delle immagini
TRAIN_SPLIT_RATIO = 0.75        # 75% dei dati originali per il training
VAL_SPLIT_RATIO = 0.15          # 15% per la validazione
# Il restante 10% sarà per il test
RANDOM_SEED = 42                # Per la riproducibilità dello split
'''

In [ ]:
#@title #### Cella 2(VECCHIA): Lo Script di Generazione

#Questa è l'unica altra cella. Fa tutto il lavoro sporco.
#
#```python
# ===================================================================
# CELLA 2: ESECUZIONE DELLA GENERAZIONE
# ===================================================================

# --- 1. PREPARA I DATI ORIGINALI ---
if not ORIGINAL_DATA_LOCAL.exists():
    ORIGINAL_DATA_LOCAL.mkdir()
    print(f"Scompatto '{ORIGINAL_ZIP_PATH}'...")
    !unzip -q -o "{ORIGINAL_ZIP_PATH}" -d "{ORIGINAL_DATA_LOCAL}"
    print("✅ Dati originali pronti.")
else:
    print(f"ℹ️ Dati originali già presenti.")

ORIGINAL_IMG_DIR = ORIGINAL_DATA_LOCAL / "images"
ORIGINAL_MSK_DIR = ORIGINAL_DATA_LOCAL / "masks"

# --- 2. DEFINISCI LE PIPELINE DI AUGMENTATION ---
# Pipeline aggressiva per il training
train_aug_pipeline = A.Compose([
    A.Resize(IMAGE_SIZE, IMAGE_SIZE),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.15, rotate_limit=25, p=0.8),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.8),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.4),
])
# Pipeline semplice (solo resize) per validazione e test
val_test_aug_pipeline = A.Compose([A.Resize(IMAGE_SIZE, IMAGE_SIZE)])

# --- 3. SPLIT DEI DATI ORIGINALI ---
all_stems = sorted([p.stem for p in ORIGINAL_IMG_DIR.glob("*.png")])
np.random.seed(RANDOM_SEED)
np.random.shuffle(all_stems)
n = len(all_stems)
n_val = int(n * VAL_SPLIT_RATIO)
n_train = int(n * TRAIN_SPLIT_RATIO)
train_ids = all_stems[:n_train]
val_ids = all_stems[n_train : n_train + n_val]
test_ids = all_stems[n_train + n_val :]
print(f"Split originale: {len(train_ids)} train, {len(val_ids)} val, {len(test_ids)} test.")

# --- 4. GENERAZIONE DEL NUOVO DATASET ---
(AUG_DATA_LOCAL / "images").mkdir(parents=True, exist_ok=True)
(AUG_DATA_LOCAL / "masks").mkdir(parents=True, exist_ok=True)
new_train_ids = []

# Processa il training set
for stem in tqdm(train_ids, desc="Augmenting Train Set"):
    img = cv2.imread(str(ORIGINAL_IMG_DIR / f"{stem}.png"))
    msk = cv2.imread(str(ORIGINAL_MSK_DIR / f"{stem}.png"), cv2.IMREAD_UNCHANGED)

    # Salva la versione originale (ridimensionata)
    resized = val_test_aug_pipeline(image=img, mask=msk)
    new_stem_orig = f"{stem}_orig"
    cv2.imwrite(str(AUG_DATA_LOCAL / "images" / f"{new_stem_orig}.png"), resized['image'])
    cv2.imwrite(str(AUG_DATA_LOCAL / "masks" / f"{new_stem_orig}.png"), resized['mask'])
    new_train_ids.append(new_stem_orig)

    # Crea e salva le versioni augmentate
    for i in range(N_AUGMENTATIONS_PER_IMAGE):
        augmented = train_aug_pipeline(image=img, mask=msk)
        new_stem_aug = f"{stem}_aug_{i}"
        cv2.imwrite(str(AUG_DATA_LOCAL / "images" / f"{new_stem_aug}.png"), augmented['image'])
        cv2.imwrite(str(AUG_DATA_LOCAL / "masks" / f"{new_stem_aug}.png"), augmented['mask'])
        new_train_ids.append(new_stem_aug)

# Processa validation e test set
for stem in tqdm(val_ids + test_ids, desc="Processing Val/Test Sets"):
    img = cv2.imread(str(ORIGINAL_IMG_DIR / f"{stem}.png"))
    msk = cv2.imread(str(ORIGINAL_MSK_DIR / f"{stem}.png"), cv2.IMREAD_UNCHANGED)
    processed = val_test_aug_pipeline(image=img, mask=msk)
    cv2.imwrite(str(AUG_DATA_LOCAL / "images" / f"{stem}.png"), processed['image'])
    cv2.imwrite(str(AUG_DATA_LOCAL / "masks" / f"{stem}.png"), processed['mask'])

# --- 5. SALVA IL NUOVO SPLIT FILE ---
final_split = {"train": new_train_ids, "val": val_ids, "test": test_ids}
with open(AUG_DATA_LOCAL / "split.json", "w") as f:
    json.dump(final_split, f, indent=4)
print(f"✅ Dataset augmentato creato in: '{AUG_DATA_LOCAL}' con {len(new_train_ids)} immagini di training.")

In [ ]:
#### Cella 2: Esempio di Utilizzo (per testare)
#Questa cella ti permette di testare la funzione direttamente nel notebook della "fabbrica".
# ===================================================================
# CELLA 2: ESECUZIONE DELLA FUNZIONE
# ===================================================================

# Questo codice viene eseguito solo se esegui questo notebook direttamente
if __name__ == '__main__':
    # Definisci i tuoi percorsi
    ZIP_FILE = "/content/drive/MyDrive/DatasetUnet/unet_dataset_multiclasse(1).zip"
    SAVE_DESTINATION = "/content/drive/MyDrive/Colab_Datasets/Augmented_Dataset_v3"

    # Chiama la funzione
    final_path = create_augmented_dataset(
        original_zip_path=ZIP_FILE,
        drive_save_path=SAVE_DESTINATION,
        n_augmentations_per_image=20 # Aumentiamo ancora per la prova!
    )

    print(f"\nProcesso terminato. Dataset disponibile in: {final_path}")

In [ ]:
!mkdir "/content/drive/MyDrive/Colab_Datasets"

In [ ]:
# --- 6. SALVA SU GOOGLE DRIVE ---
print(f"Salvataggio del dataset augmentato su Google Drive in: '{DRIVE_SAVE_PATH}'...")
!cp -r "{AUG_DATA_LOCAL}" "{DRIVE_SAVE_PATH}"
print("✅ Lavoro completato. La fabbrica si spegne.")